In [1]:
import pandas as pd
import os
import shutil
import glob

In [2]:
# Read file processed_anno_original.csv
df = pd.read_csv('annotations/processed_anno_original.csv',sep=' ',names=['fileName',"labelClass"], header=None,index_col=False)
#Output df: 1115 rows

#Remove row that labelClass = -1, or -2
df = df.drop(df[df.labelClass < 0].index)
#Output df: 537 rows

#Remove duplicate values
df = df.drop_duplicates()
#Output df: 537 rows
df.reset_index(drop=True, inplace=True)

In [3]:
# Create a column: camera view
df.loc[df['fileName'].str.contains('Dashboard') == True, 'cView'] = "1"
df.loc[df['fileName'].str.contains('Right') == True, 'cView'] = "2"
df.loc[df['fileName'].str.contains('Rear') == True, 'cView'] = "3"

In [4]:
new = df['fileName'].str.split(".", n = 4, expand = True)

In [5]:
df["userID"] = new[1]
df["startTime"] = new[2]
df["endTime"] = new[3]

In [6]:
old_cols = df.columns.values 
old_cols

array(['fileName', 'labelClass', 'cView', 'userID', 'startTime',
       'endTime'], dtype=object)

In [7]:
new_cols= ['fileName', 'startTime', 'endTime', 'userID','cView','labelClass']
df = df.reindex(columns=new_cols)

In [8]:
df

,fileName,startTime,endTime,userID,cView,labelClass
0,Rightside_window_user_id_24491_NoAudio_1.24491...,18,45,24491,2,3
1,Rightside_window_user_id_24491_NoAudio_1.24491...,45,54,24491,2,14
2,Rightside_window_user_id_24491_NoAudio_1.24491...,74,105,24491,2,2
3,Rightside_window_user_id_24491_NoAudio_1.24491...,113,117,24491,2,8
4,Rightside_window_user_id_24491_NoAudio_1.24491...,137,166,24491,2,5
...,...,...,...,...,...,...
532,Right_window_user_id_49381_NoAudio_1.49381.401...,401,425,49381,2,16
533,Right_window_user_id_49381_NoAudio_1.49381.435...,435,451,49381,2,10
534,Right_window_user_id_49381_NoAudio_1.49381.461...,461,478,49381,2,5
535,Right_window_user_id_49381_NoAudio_1.49381.488...,488,503,49381,2,11


In [9]:
# g = df.groupby(['labelClass', df.cView])
g = df.groupby(['labelClass', df.cView])

In [15]:
for (labelClass, cView), group in g:
    if not os.path.exists("datasets/" + str(labelClass)):
        os.makedirs("datasets/" + str(labelClass))
    group.iloc[:,:].to_csv(f'{"datasets/" + str(labelClass)}/{cView}.csv', index=False)

for i in range(18):    
    interesting_files = glob.glob("datasets/"+ str(i) +"/*.csv") 
    df_concat = pd.concat((pd.read_csv(f, header = 0) for f in interesting_files))
    df_deduplicated = df_concat.drop_duplicates()
    df_deduplicated.to_csv("datasets/" + str(i) + "/a" + str(i) + ".csv",index=False)

df_final = list()    
for i in range(18):   
    join_file = pd.read_csv("datasets/"+ str(i) + "/a" + str(i) + ".csv")
    df_final.append(join_file)
dffinal = pd.concat(df_final, ignore_index=True)
dffinal = dffinal.drop_duplicates()
dffinal.to_csv("datasets/" + "full_video.csv",index=False)

# Remove all csv in folder 0 - 17
for i in range(18):   
    dir_path = "datasets/" + str(i)
    dir_list = os.listdir(dir_path)
    for item in dir_list:
        if item.endswith(".csv"):
            os.remove(os.path.join(dir_path, item))

In [16]:
# # Create folder camera view
# import os
# for (cView, labelClass), group in g:
#     if not os.path.exists("annotations/camView" + str(cView)):
#         os.makedirs("annotations/camView" + str(cView))
#     group.iloc[:,:].to_csv(f'{"annotations/camView" + str(cView)}/{labelClass}.csv', index=False)

In [30]:
data = pd.read_csv("datasets/" + "full_video.csv")
for row in range(len(data)):
    source_file = os.path.join("A1_clips/"+str(data["fileName"][row]))
    dest_file = os.path.join("datasets" + "/" + str(data["labelClass"][row]) + "/" + str(data["fileName"][row]))
    shutil.copy(source_file, dest_file)
    
data["fileName"] = "datasets/" + data["labelClass"].astype(str) +"/" + data["fileName"].astype(str) + " " + data["labelClass"].astype(str)
data = data.drop(['startTime','endTime','userID','cView','labelClass'], axis=1)
data.to_csv("datasets/" + "final_videos.csv",header=False, index=False)

In [31]:
import pandas

# df = pd.read_csv('total_data.csv')
k_map = {"dashboard": "Dashboard", "rearview": "Rear", "right": "Right"}

for view in ["dashboard", "rearview", "right"]:
    for user_id in ["49381", "38058", "35133", "24491", "24026"]:
        f = open("datasets/final_videos.csv", "r")
        f_train = open("datasets/train_{}_{}.csv".format(view, user_id), "w")
        f_val = open("datasets/val_{}_{}.csv".format(view, user_id), "w")
        f_test = open("datasets/test_{}_{}.csv".format(view, user_id), "w")
        for line in f.readlines():
            if k_map[view] in line:
                if user_id in line:
                    print(line)
                    f_val.write(line)
                    f_test.write(line)
                else:
                    f_train.write(line)
        f_val.close()
        f_train.close()
        f_test.close()
        f.close()

datasets/0/Dashboard_user_id_49381_NoAudio_0.49381.98.120.MP4 0

datasets/0/Dashboard_user_id_49381_NoAudio_1.49381.107.129.MP4 0

datasets/1/Dashboard_user_id_49381_NoAudio_0.49381.42.62.MP4 1

datasets/1/Dashboard_user_id_49381_NoAudio_1.49381.49.69.MP4 1

datasets/2/Dashboard_user_id_49381_NoAudio_0.49381.504.526.MP4 2

datasets/2/Dashboard_user_id_49381_NoAudio_1.49381.513.535.MP4 2

datasets/3/Dashboard_user_id_49381_NoAudio_0.49381.15.32.MP4 3

datasets/3/Dashboard_user_id_49381_NoAudio_1.49381.20.39.MP4 3

datasets/4/Dashboard_user_id_49381_NoAudio_0.49381.272.294.MP4 4

datasets/4/Dashboard_user_id_49381_NoAudio_1.49381.281.303.MP4 4

datasets/5/Dashboard_user_id_49381_NoAudio_0.49381.452.469.MP4 5

datasets/5/Dashboard_user_id_49381_NoAudio_1.49381.461.478.MP4 5

datasets/6/Dashboard_user_id_49381_NoAudio_0.49381.72.88.MP4 6

datasets/6/Dashboard_user_id_49381_NoAudio_1.49381.80.97.MP4 6

datasets/7/Dashboard_user_id_49381_NoAudio_0.49381.162.186.MP4 7

datasets/7/Dashboard_us